## Cleaning Data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('murder_data.csv')
df = pd.DataFrame(data)

In [3]:
df.head()

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,fstate,MSA
0,197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,ALASKA,...,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180,Alaska,Rural Alaska
1,197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,ALA,...,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180,Alabama,"Birmingham-Hoover, AL"
2,197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,ALA,...,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180,Alabama,"Birmingham-Hoover, AL"
3,197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,ALA,...,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180,Alabama,"Birmingham-Hoover, AL"
4,197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,ALA,...,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180,Alabama,"Mobile, AL"


In [11]:
# Change VicAge string values to integers
df['VicAge'] = df['VicAge'].replace('Newborn or infant', 1)
df['VicAge'] = df['VicAge'].replace('Age unknown', 0)
df['VicAge'] = df['VicAge'].replace('99 years old and more', 99)
# Change OffAge string values to integers
df['OffAge'] = df['OffAge'].replace('Offender unknown', 0)
df['OffAge'] = df['OffAge'].replace('99 years old and more', 99)

In [14]:
df.to_csv('clean_murder_data.csv', index=False)